# Features extraction

In this file, the pre-trained model (https://github.com/csho33/bacteria-ID) is used as a features extractor.

## Librairies

In [1]:
from data_loader import load_data

## Dataset

In [2]:
X, Y, groups = load_data()